In [57]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
import itertools as it
import missing_bayesian_mixture as mbm
import os
import denoising_ae
import category_dae
import tensorflow as tf
import missForest
import missForest_GP

from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(mbm)
imp.reload(category_dae)
imp.reload(missForest)
imp.reload(missForest_GP)
imp.reload(denoising_ae)

<module 'denoising_ae' from '/home/adria/MIMIC/ts-baselines/timeless-imputation/denoising_ae.py'>

In [3]:
from IPython.display import HTML
def df_view(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))

In [27]:
dsets = datasets.datasets()

+++ Importing Soybean
+++ Importing Servo
+++ Importing BreastCancer
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing LetterRecognition
+++ Importing BostonHousing
+++ Importing Shuttle


In [54]:
_ds = dict((x, dsets[x]) for x in ["BostonHousing", "Servo", "Ionosphere"
    # "Soybean", "BreastCancer"
])
baseline = datasets.benchmark({#'MICE': datasets.memoize(utils.impute_mice),
        #'MF_py': missForest.impute,
        'mean': missForest.impute,
        'GMM': missForest.impute,
        'MissForest': datasets.memoize(utils.impute_missforest),
        'GP': missForest.impute,
        'GP_ARD': missForest.impute,
        'GP_unopt': missForest.impute,
        'GP_uncertain': missForest.impute,

    }, _ds, do_not_compute=True)

In [61]:
baseline.loc[("mean_std"), "NRMSE"]# .drop([("NRMSE", "MF_py"), ("PFC", "MF_py"), ("RMSE", "MF_py")], axis=1)  # .drop(["LetterRecognition", "Shuttle", "Soybean"], axis=0)

GMM        GP    GP_ARD  GP_uncertain  \
BostonHousing MCAR_rows  0.1  0.258737       NaN       NaN           NaN   
                         0.3  0.265258       NaN       NaN           NaN   
                         0.5  0.241586       NaN       NaN           NaN   
                         0.7  0.308484       NaN       NaN           NaN   
                         0.9  0.367254  0.372170       NaN           NaN   
              MCAR_total 0.1  0.199314  0.212968  0.212908      0.250720   
                         0.3  0.171907       NaN       NaN           NaN   
                         0.5  0.207475  0.216506       NaN      0.305964   
                         0.7  0.275221       NaN       NaN           NaN   
                         0.9  0.349374  0.383940       NaN      0.373346   
Ionosphere    MCAR_rows  0.1  0.824452       NaN       NaN           NaN   
                         0.3  0.848767  0.706740       NaN      0.893286   
                         0.5  0.878189       NaN       NaN           NaN   
                         0.7  0.877579  0.889716       NaN      0.929054   
                         0.9  0.981320  0.946039       NaN      0.935541   
              MCAR_total 0.1  0.863302       NaN       NaN           NaN   
                         0.3  0.831159  0.698453       NaN           NaN   
                         0.5  0.852060       NaN       NaN           NaN   
                         0.7  0.889136       NaN       NaN           NaN   
                         0.9  0.993140       NaN       NaN           NaN   
Servo         MCAR_rows  0.1       NaN       NaN       NaN           NaN   
                         0.3       NaN       NaN       NaN           NaN   
                         0.5       NaN       NaN       NaN           NaN   
                         0.7       NaN       NaN       NaN           NaN   
                         0.9       NaN       NaN       NaN           NaN   
              MCAR_total 0.1       NaN       NaN       NaN           NaN   
                         0.3       NaN       NaN       NaN           NaN   
                         0.5       NaN       NaN       NaN           NaN   
                         0.7       NaN       NaN       NaN           NaN   
                         0.9       NaN       NaN       NaN           NaN   

                              GP_unopt     MF_py  MissForest      mean  
BostonHousing MCAR_rows  0.1       NaN  0.233296    0.229694  0.406591  
                         0.3       NaN  0.267079    0.268958  0.402195  
                         0.5       NaN  0.242708    0.236956  0.370293  
                         0.7       NaN  0.309237    0.310009  0.376315  
                         0.9  0.378890  0.378195    0.386043  0.381834  
              MCAR_total 0.1  0.241800  0.174714    0.171257  0.366148  
                         0.3       NaN  0.167097    0.169159  0.358658  
                         0.5  0.254300  0.205371    0.205258  0.376983  
                         0.7       NaN  0.273668    0.263226  0.378586  
                         0.9  0.379549  0.408480    0.369981  0.378147  
Ionosphere    MCAR_rows  0.1       NaN  0.630301    0.623871  0.915913  
                         0.3  0.861571  0.649127    0.645065  0.909305  
                         0.5       NaN  0.713889    0.708755  0.927602  
                         0.7  0.904933  0.830069    0.818316  0.936444  
                         0.9  0.935980  1.007278    0.943462  0.938464  
              MCAR_total 0.1  0.865501  0.680204    0.679802  0.937527  
                         0.3  0.901823  0.659374    0.660242  0.942459  
                         0.5       NaN  0.685098    0.683843  0.930471  
                         0.7       NaN  0.773375    0.751468  0.932162  
                         0.9       NaN  0.989135    1.032203  0.939575  
Servo         MCAR_rows  0.1       NaN       NaN         NaN       NaN  
                         0.3       NaN       NaN         NaN       NaN  

In [170]:
imp.reload(mbm)
bayesgmm = datasets.benchmark({'BGMM_50': (lambda p, d, full_data: mbm.impute_bayes_gmm(
    p, d, full_data=full_data, number_imputations=100, n_components=50))
                              }, dsets, do_not_compute=True)

In [295]:
results = pd.concat([baseline, bayesgmm, nnresults], axis=1).loc['mean_std']
results = results.drop([('total_cats', 'MissForest'), ('total_cats', 'MLP_2x256_SGD')], axis=1)
results.sort_index(level=[0,1], inplace=True, axis=1)
results

In [319]:
# Export the results to LaTeX

s = (("""\\documentclass[12pt,a3paper,landscape,oneside]{article}
\\usepackage[utf8]{inputenc}
\\usepackage[english]{babel}
\\usepackage{setspace}
\\usepackage{parskip}
\\usepackage{color}
\\usepackage{amssymb,amsmath,amsfonts}
\\usepackage{graphicx}
\\usepackage{hyperref}
\\hypersetup{colorlinks,breaklinks,urlcolor=blue,linkcolor=blue} % Set link colors throughout the document
\\usepackage[left=2cm,right=2cm,top=2cm,bottom=2cm]{geometry}
\\usepackage{enumitem}   
\\usepackage{import}
\\usepackage{multicol}

\\usepackage{csquotes}
\\begin{document}\n""" + results.to_latex() + "\n\\end{document}")
     .replace("\\toprule", "\\hline")
     .replace("\\bottomrule", "\\hline")
     .replace("\\midrule", "\\hline"))

with open("measurements.tex", "w") as f:
    f.write(s)


In [ ]:
from rpy2.robjects.packages import importr
import rpy2.robjects
import rpy2.robjects.pandas2ri
import rpy2.robjects.numpy2ri
import rpy2.rinterface
import numpy as np
import pickle_utils as pu
import pandas as pd
import collections
import utils
import os
import collections
import unittest

rpy2.robjects.pandas2ri.activate()
rpy2.robjects.numpy2ri.activate()
py2ri = rpy2.robjects.pandas2ri.py2ri
ri2py = rpy2.robjects.pandas2ri.ri2py
R = rpy2.robjects.r

base = importr("base")
R_utils = importr("utils")
mlbench = importr("mlbench")
randomForest = importr("randomForest")

In [77]:
dsets_label = datasets.datasets(exclude_labels=False)
path = "impute_benchmark"
for algo_name in ["BayesGMM", "MICE", "MissForest"]:
    for data_name, (full_data, cat_keys) in dsets_label.items():
        if data_name == 'BostonHousing':
            continue

        for ampute_fun_name in ['MCAR_total', 'MCAR_rows']:
            for proportion in [.1, .3, .5, .7, .9]:
                for norm_type in ['mean_std']:  # 'min_max']:
                    amputed_name = '{:s}_{:s}_{:.1f}'.format(
                        data_name, ampute_fun_name, proportion) 
                    imputed_path = os.path.join(
                        path, 'imputed_{:s}_{:s}.pkl.gz'.format(
                            algo_name, amputed_name))
                    if not os.path.exists(imputed_path):
                        continue
                    amputed_data = pu.load(os.path.join(path, 'amputed_'+amputed_name+'.pkl.gz'))
                    imputed_data = pu.load(imputed_path)
                    (_ad, _id), moments = utils.normalise_dataframes(
                        amputed_data, imputed_data, method=norm_type)
                    
                    to_learn = pd.concat([_id, full_data[["Class"]]], axis=1)
                    test_mask = np.random.rand(len(to_learn)) < 0.2
                    training_set = to_learn[~test_mask]
                    test_set = to_learn[test_mask]
                    
                    


+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing BostonHousing


AssertionError: All dataframes must have the same columns